## DPLA API Interactions

Code to interact with the DPLA API. Designed specifically to produce output suited to further, manual data entry, for eventual conversion to RDF according to the [Collex standard](http://wiki.collex.org/index.php/Main_Page).

In [1]:
from dpla_api import DplaApi
da = DplaApi()
# Run search, page_size is capped (by DPLA) at 500, but code will iterate through 
# as many pages as necessary to gather all results.
da.search({"sourceResource.title" : "crime", "sourceResource.spatial.state" : "Illinois"}, page_size=500) 
# All retrieved results will be stored in the da.metadata_records object.
da.build_arc_rdf_dataset(check_match=False)
da.create_tsv()

Query: '{'sourceResource.spatial.state': 'Illinois', 'sourceResource.title': 'crime'}' returned 67 results
----Check: 67 records transferred
Completed writing data/radicalism-dpla.tsv


In [3]:
da.metadata_records

[{'archive': '',
  'creator': [],
  'date': '',
  'discipline': '',
  'federation': 'SiRO',
  'genre': 'none',
  'id': u'http://dp.la/api/items/3b83fb0f03198bee0092939cf2d08a64',
  'language': [u'English'],
  'original_query': {'sourceResource.spatial.state': 'Illinois',
   'sourceResource.title': 'crime'},
  'role': '',
  'seeAlso': u'http://www.idaillinois.org/cdm/ref/collection/stc/id/14257',
  'source': u'Illinois Digital Heritage Hub',
  'subjects': [u'Saint Charles (Ill.)--History',
   u'Crime--Illinois--Saint Charles'],
  'thumbnail': u'http://www.idaillinois.org/utils/getthumbnail/collection/stc/id/14257',
  'title': u'Crime',
  'type': u'text'},
 {'archive': '',
  'creator': [],
  'date': u'1971',
  'discipline': '',
  'federation': 'SiRO',
  'genre': 'none',
  'id': u'http://dp.la/api/items/0d0f53f4a703d19a345b1447f3d0b1a8',
  'language': [u'English'],
  'original_query': {'sourceResource.spatial.state': 'Illinois',
   'sourceResource.title': 'crime'},
  'role': '',
  'seeAls

The code below is an implementation for running a specific set of searches drawn from a json file.

In [ ]:
from dpla_api import DplaApi
import json
da = DplaApi()
with open("mich-results.json") as f:
    data = json.load(f)
da.create_tsv(records=data)

In [ ]:
import json
with open("mich-results.json", "w") as f:
    json.dump(da.metadata_records, f)

In [ ]:
from dpla_api import DplaApi
import json
da = DplaApi()
all_metadata = []
search_file = "data/dpla_subjects.json"
with open(search_file) as f:
    search_terms = json.load(f)
# Disciplines, in this case, is an extra data point useful for creating
# the brand of RDF used in the SiRO project.
for search_term, disciplines in search_terms.items():
    if len(da.metadata_records) > 10000:
        break
    else:
        da.search(search_term, page_size=500)
        if da.result.count > 0:
            da.build_arc_rdf_dataset(disciplines=disciplines)
            # all_metadata += da.metadata_records
# Create a TSV file of output for closer analysis / update of terms.
da.create_tsv(records=da.metadata_records)

Once complete with any necessary changes, the tsv file can be used to create RDF records, 1 for each record.

In [3]:
from buildrdf import BuildRdf
br = BuildRdf()
br.build_rdf_from_tsv("data/Labadie 2017-06-16 - Sheet1 (3).tsv")

Processing 3026 records...
Processed 0 records
Processed 500 records
Processed 1000 records
Processed 1500 records
Processed 2000 records
Processed 2500 records
Processed 3000 records


In order to make sure each successive bath of DPLA results doesn't simply reintroduce the results of any previous batches, there is a listing of all previously encountered results stored in a JSON file. An on-the-fly version is updated with searches as they complete, _however_ this cache should be cleared if these search results don't wind up making it into RDF records, by using the `reset_matches` parameter below.

In [ ]:
from dpla_api import DplaApi
da = DplaApi()
da.update_rdf_registry(reset_matches=True)

In [ ]:
# Build json file of search terms.
import json
subject_file = "data/estc_subjects.tsv"
subject_dict = {}
with open(subject_file) as f:
    for line in f:
        values = line.split("\t")
        if values[1] == "x":
            subject_dict[values[0]] = values[2]
with open("data/dpla_subjects.json", "w") as g:
    json.dump(subject_dict, g)

In [ ]:
da._marc_record(da.all_returned_items[0])

In [ ]:
for item in da.all_returned_items:
    if da._get_genre_from_marc(item) != "none":
        print da._get_genre_from_marc(item)